# Анализ производетльности такси в Нью-Йорке

Конечно! Это один из датасетов NYC Taxi & Limousine Commission (TLC) - данные о поездках в желтых такси Нью-Йорка. Вот подробное описание каждого поля:

## Основная информация о поездке

**`vendorid`** - Идентификатор провайдера/вендора такси
- 1 = Creative Mobile Technologies (CMT)
- 2 = VeriFone Inc. (VTS)

**`tpep_pickup_datetime`** - Дата и время начала поездки (Timestamp)

**`tpep_dropoff_datetime`** - Дата и время окончания поездки (Timestamp)

**`passenger_count`** - Количество пассажиров в такси

## Географическая информация

**`trip_distance`** - Расстояние поездки в милях

**`pulocationid`** - ID зоны такси (Taxi Zone) для посадки
- Ссылается на таблицу taxi zones

**`dolocationid`** - ID зоны такси (Taxi Zone) для высадки
- Ссылается на таблицу taxi zones

**`ratecodeid`** - Тип тарифа:
- 1 = Стандартный тариф
- 2 = JFK Airport
- 3 = Newark Airport
- 4 = Nassau or Westchester
- 5 = Negotiated fare
- 6 = Group ride

## Флаги и статусы

**`store_and_fwd_flag`** - Флаг хранения и пересылки данных
- Y = поездка была сохранена в памяти устройства и передана позже
- N = поездка передана в реальном времени

**`payment_type`** - Тип оплаты:
- 1 = Кредитная карта
- 2 = Наличные
- 3 = Без оплаты
- 4 = Спор
- 5 = Неизвестно
- 6 = Voided trip

## Стоимость поездки (в долларах)

**`fare_amount`** - Основная стоимость проезда

**`extra`** - Дополнительные платежи:
- Пиковое время, ночные надбавки и т.д.

**`mta_tax`** - Налог MTA ($0.50)

**`tip_amount`** - Чаевые

**`tolls_amount`** - Плата за проезд по платным дорогам/мостам

**`improvement_surcharge`** - Надбавка за улучшение сервиса ($0.30)

**`total_amount`** - Общая сумма к оплате

**`congestion_surcharge`** - Надбавка за пробки
- Применяется в зонах с высокой загруженностью

**`airport_fee`** - Аэропортовый сбор
- При поездках из аэропортов

## Особенности данных:

1. **Частота обновления**: Ежемесячно
2. **Формат**: Parquet (оптимизирован для аналитики)
3. **Объем**: Обычно несколько миллионов записей в месяц
4. **Качество**: Официальные данные от TLC, но требуют очистки

## Типичные проблемы данных:
- Отрицательные стоимости
- Нулевые расстояния при ненулевой стоимости
- Аномальные значения времени поездки
- Пропущенные значения в некоторых полях

Эти данные широко используются для:
- Анализа транспортных потоков
- Прогнозирования спроса на такси
- Геопространственного анализа
- Анализа доходов и бизнес-метрик

Хочешь, чтобы я помог проанализировать конкретные аспекты этого датасета?

Отличные вопросы! Этот датасет очень богат на возможности для анализа и машинного обучения.

## 📊 **Метрики для анализа**

### **Временные метрики**
```python
# Ежедневная/ежемесячная активность
daily_trips = df.groupBy(date_format("tpep_pickup_datetime", "yyyy-MM-dd")).count()

# Часы пиковой нагрузки
peak_hours = df.groupBy(hour("tpep_pickup_datetime")).count().orderBy("hour")
```

### **Географические метрики**
```python
# Самые популярные зоны посадки/высадки
top_pickup_locations = df.groupBy("pulocationid").count().orderBy(desc("count"))
top_dropoff_locations = df.groupBy("dolocationid").count().orderBy(desc("count"))
```

### **Финансовые метрики**
```python
# Средний чек по часам/дням
avg_fare_by_hour = df.groupBy(hour("tpep_pickup_datetime")).agg(
    avg("total_amount").alias("avg_revenue")
)

# Распределение типов оплат
payment_distribution = df.groupBy("payment_type").count()
```

### **Операционные метрики**
```python
# Средняя длительность и расстояние поездок
df.withColumn("trip_duration_minutes", 
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime"))/60
).select(
    avg("trip_duration_minutes").alias("avg_duration"),
    avg("trip_distance").alias("avg_distance")
).show()
```

## 🤖 **Машинное обучение на этом датасете**

### **1. Предсказание длительности поездки**
**Вопрос**: "Сколько времени займет поездка из точки A в точку B?"
```python
# Целевая переменная
df = df.withColumn("trip_duration", 
    unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")
)

# Признаки: день недели, час, зоны, расстояние, пассажиры
```

### **2. Предсказание стоимости поездки**
**Вопрос**: "Сколько будет стоить поездка?"
```python
# Целевая переменная: total_amount
# Признаки: расстояние, время суток, день недели, зоны, пробки
```

### **3. Классификация чаевых**
**Вопрос**: "Даст ли пассажир чаевые?"
```python
# Бинарная классификация
df = df.withColumn("has_tip", when(col("tip_amount") > 0, 1).otherwise(0))
```

### **4. Прогнозирование спроса**
**Вопрос**: "Сколько поездок будет в следующем часу/дне?"
```python
# Временные ряды - агрегация по часам
hourly_demand = df.groupBy(
    hour("tpep_pickup_datetime").alias("hour"),
    dayofweek("tpep_pickup_datetime").alias("day_of_week")
).count()
```

### **5. Кластеризация поездок**
**Вопрос**: "Какие типы поездок существуют?"
```python
# Кластеризация по продолжительности, расстоянию, стоимости
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=["trip_distance", "total_amount", "passenger_count"],
    outputCol="features"
)
```

## 💡 **Конкретные бизнес-вопросы для ML**

### **Для такси-компаний:**
- "В какие зоны направить такси в следующий час?"
- "Какие поездки наиболее прибыльные?"
- "Как погода влияет на спрос?"

### **Для пассажиров:**
- "Когда лучше ехать, чтобы избежать пробок?"
- "Сколько в среднем стоит поездка в мой район?"

### **Для городского планирования:**
- "Где построить новые такси-стоянки?"
- "Какие районы нуждаются в улучшении транспорта?"

## 🛠 **Пример пайплайна ML:**
```python
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler, StringIndexer

# Подготовка признаков
df_ml = df.withColumn("trip_duration", 
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime"))/60
).filter(
    (col("trip_duration") > 0) & (col("trip_duration") < 180)  # фильтрация аномалий
)

# Индексация категориальных признаков
zone_indexer = StringIndexer(inputCol="pulocationid", outputCol="pu_zone_index")
hour_indexer = StringIndexer(inputCol=hour("tpep_pickup_datetime"), outputCol="hour_index")

# Векторизация признаков
assembler = VectorAssembler(
    inputCols=["pu_zone_index", "hour_index", "trip_distance", "passenger_count"],
    outputCol="features"
)

# Модель
rf = RandomForestRegressor(featuresCol="features", labelCol="trip_duration")
```

Этот датасет - золотая жила для Data Scientist! Что именно тебя интересует больше - анализ или предсказание?

Отличный вопрос! Кластеризация поездок может раскрыть скрытые паттерны в данных. Давайте разберем подробно.

## 🎯 **Цели кластеризации поездок**

### **Что мы можем узнать:**
- Типичные сценарии поездок
- Аномальные/подозрительные поездки
- Сегменты клиентов
- Оптимизацию бизнес-процессов

## 🔧 **Практическая реализация**

```python
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql.functions import *

# Подготовка данных
df_cluster = df.withColumn("trip_duration_minutes", 
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60
).filter(
    (col("trip_duration_minutes") > 0) & 
    (col("trip_duration_minutes") < 180) &
    (col("trip_distance") > 0) &
    (col("trip_distance") < 50) &
    (col("total_amount") > 0) &
    (col("total_amount") < 200)
)

# Выбор признаков для кластеризации
assembler = VectorAssembler(
    inputCols=["trip_distance", "trip_duration_minutes", "total_amount", "passenger_count"],
    outputCol="raw_features"
)

# Масштабирование признаков (важно для K-means!)
scaler = StandardScaler(
    inputCol="raw_features",
    outputCol="features",
    withStd=True,
    withMean=True
)

# Кластеризация
kmeans = KMeans(
    featuresCol="features",
    predictionCol="cluster",
    k=5,  # количество кластеров
    seed=42
)

# Пайплайн
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[assembler, scaler, kmeans])
model = pipeline.fit(df_cluster)

# Предсказание
result = model.transform(df_cluster)
```

## 📊 **Анализ результатов кластеризации**

### **1. Статистика по кластерам**
```python
# Базовая статистика по кластерам
cluster_stats = result.groupBy("cluster").agg(
    count("*").alias("count"),
    avg("trip_distance").alias("avg_distance"),
    avg("trip_duration_minutes").alias("avg_duration"),
    avg("total_amount").alias("avg_amount"),
    avg("passenger_count").alias("avg_passengers"),
    percentile_approx("total_amount", 0.5).alias("median_amount")
).orderBy("cluster")

cluster_stats.show()
```

### **2. Визуализация центроидов**
```python
# Получаем центры кластеров
centers = model.stages[-1].clusterCenters()

print("Центры кластеров:")
for i, center in enumerate(centers):
    print(f"Кластер {i}: {center}")
```

## 🔍 **Интерпретация кластеров**

### **Типичные паттерны, которые можно обнаружить:**

#### **Кластер 0: "Короткие городские поездки"**
```
- Среднее расстояние: 1-3 мили
- Длительность: 5-15 минут
- Стоимость: $8-15
- Пассажиры: 1-2
```
**Интерпретация**: Быстрые поездки по центру города, возможно, деловые встречи

#### **Кластер 1: "Аэропортовые поездки"**
```
- Среднее расстояние: 15-25 миль  
- Длительность: 30-60 минут
- Стоимость: $50-80
- Пассажиры: 1-2 с багажом
```
**Интерпретация**: Поездки в JFK/LaGuardia

#### **Кластер 2: "Дальние поездки"**
```
- Среднее расстояние: 8-12 миль
- Длительность: 25-40 минут
- Стоимость: $30-50
- Пассажиры: 1-3
```
**Интерпретация**: Поездки между районами, возможно, в пригород

#### **Кластер 3: "Групповые поездки"**
```
- Среднее расстояние: 2-5 миль
- Длительность: 10-20 минут
- Стоимость: $12-25
- Пассажиры: 4-6
```
**Интерпретация**: Групповые поездки, возможно, туристы

#### **Кластер 4: "Ночные поездки"**
```
- Среднее расстояние: 3-7 миль
- Длительность: 15-30 минут  
- Стоимость: $20-40 (с ночными надбавками)
- Пассажиры: 2-3
```
**Интерпретация**: Поездки вечером/ночью

## 💡 **Бизнес-инсайты из кластеризации**

### **Для оптимизации бизнеса:**
```python
# Анализ прибыльности кластеров
profitability = result.groupBy("cluster").agg(
    avg("total_amount").alias("avg_revenue"),
    avg("trip_duration_minutes").alias("avg_duration"),
    (avg("total_amount") / avg("trip_duration_minutes")).alias("revenue_per_minute")
).orderBy(desc("revenue_per_minute"))

profitability.show()
```

### **Что можно узнать:**

1. **Самые прибыльные сегменты** - какие типы поездок приносят больше денег в минуту
2. **Оптимизация флота** - куда направлять такси в разное время суток
3. **Ценообразование** - понимание ценовых сегментов
4. **Аномалии** - выявление мошеннических или ошибочных поездок
5. **Сезонность** - как паттерны меняются по дням недели/времени года

## 🚨 **Обнаружение аномалий**

```python
# Поездки, которые не вписываются в кластеры
anomalies = result.filter(
    (col("trip_distance") > 50) | 
    (col("total_amount") > 200) |
    (col("trip_duration_minutes") > 180)
)

print(f"Обнаружено аномальных поездок: {anomalies.count()}")
```

## 📈 **Визуализация результатов**

```python
# Для визуализации можно преобразовать в pandas
cluster_sample = result.select(
    "trip_distance", "trip_duration_minutes", "total_amount", "cluster"
).sample(0.1).toPandas()

# Scatter plot кластеров
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 8))
sns.scatterplot(data=cluster_sample, x="trip_distance", y="total_amount", hue="cluster", palette="viridis")
plt.title("Кластеризация поездок такси")
plt.xlabel("Расстояние (мили)")
plt.ylabel("Стоимость ($)")
plt.show()
```

## 🎯 **Практическое применение результатов**

**Для такси-компании:**
- "Кластер 1 (аэропорты) - самый прибыльный, увеличить количество машин у аэропортов"
- "Кластер 3 (групповые) - низкая маржинальность, возможно ввести спецтарифы"
- "Кластер 4 (ночные) - высокие надбавки, стимулировать водителей работать ночью"

**Для городского планирования:**
- "Кластер 0 показывает высокий спрос в центре - нужны дополнительные стоянки"
- "Кластер 2 выявляет потребность в улучшении транспорта между районами"

Кластеризация превращает сырые данные в осмысленные бизнес-инсайты!

## 1. Получение данных в слой bronze
В качестве датасета возьмем данные поездок желтого такси. Для этого зафармим файлы отсюда: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

Создадим бакет `bronze` в `MinIO` и напишем функции которые позволят докачивать файлы с учетом уже имеющихся


In [1]:
from minio import Minio
from minio.error import S3Error
import requests
from tqdm import tqdm
import os
import tempfile


# -------------------- Проверить какие файлы фактически существуют на сайте --------------------

def get_available_remote_files(base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                               filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                               year=2024):
    """Проверить какие файлы фактически существуют на сайте"""
    available_files = []
    
    print("🔍 Проверка доступных файлов на сайте...")
    
    for month in tqdm(range(1, 13), desc="Проверка месяца"):
        filename = filename_template.format(year=year, month=month)
        url = f"{base_url}/{filename}"
        
        try:
            # Делаем HEAD запрос чтобы проверить существование файла
            response = requests.head(url, timeout=10)
            if response.status_code == 200:
                available_files.append(filename)
                print(f"  ✓ {filename} - доступен")
            else:
                print(f"  ✗ {filename} - недоступен (код: {response.status_code})")
                
        except requests.exceptions.RequestException as e:
            print(f"  ✗ {filename} - ошибка: {e}")
    
    return available_files

# -------------------- Получить список файлов в MinIO --------------------

def get_local_minio_files(minio_client, 
                          bucket_name="bronze", 
                          prefix="nyc-taxi-data"):
    """Получить список файлов в MinIO"""
    local_files = []
    try:
        objects = minio_client.list_objects(bucket_name, prefix=prefix, recursive=True)
        for obj in objects:
            # Извлекаем имя файла из полного пути
            filename = obj.object_name.replace(f"{prefix}/", "")
            local_files.append(filename)
    except S3Error as e:
        print(f"Ошибка при чтении бакета: {e}")
    
    return local_files

# -------------------- Загрузка только отсутствующих файлов в MinIO --------------------  

def download_missing_files(bucket_name="bronze", 
                           prefix="nyc-taxi-data",
                           base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                           filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                           year=2024):
    """Загрузка только отсутствующих файлов в MinIO"""
    # Настройка клиента MinIO
    minio_client = Minio(
        "minio:9000",
        access_key="minioadmin",
        secret_key="minioadmin",
        secure=False
    )
    
    # Создаем бакет если нужно
    try:
        if not minio_client.bucket_exists(bucket_name):
            minio_client.make_bucket(bucket_name)
            print(f"✓ Бакет {bucket_name} создан")
    except S3Error as e:
        return [f"✗ Ошибка бакета: {e}"]
    
    # Получаем списки файлов
    remote_files = get_available_remote_files(base_url, filename_template, year)
    local_files = get_local_minio_files(minio_client, bucket_name, prefix)
    
    # Находим отсутствующие файлы
    missing_files = list(set(remote_files) - set(local_files))

    # Блок статистики
    print(f"\n📊 СТАТИСТИКА:")

    print(f"• Загружено в MinIO: {len(local_files)} файл(ов)")    

    print()    
    print(f"• Доступно на сайте: {len(remote_files)} файл(ов)")
    for file in sorted(remote_files):
        print(f"     - {file}")

    print()
    if missing_files:
        print(f"• Из них отсутствует в MinIO: {len(missing_files)} файл(ов)")
        for file in sorted(missing_files):
            print(f"     - {file}")
    
    if not missing_files:
        return ["✅ Все доступные файлы уже загружены"]
    
    results = []
    
    # Скачиваем только отсутствующие файлы
    for filename in tqdm(missing_files, desc="Загрузка недостающих"):
        url = f"{base_url}/{filename}"
        
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            
            # Создаем временный файл
            with tempfile.NamedTemporaryFile(delete=False, suffix='.parquet') as temp_file:
                temp_path = temp_file.name
                
                # Скачиваем файл на диск
                total_size = int(response.headers.get('content-length', 0))
                for chunk in response.iter_content(chunk_size=8192 * 8):
                    if chunk:
                        temp_file.write(chunk)
            
            # Получаем реальный размер файла
            file_size = os.path.getsize(temp_path)
            
            # Загружаем в MinIO
            minio_client.fput_object(
                bucket_name=bucket_name,
                object_name=f"{prefix}/{filename}",
                file_path=temp_path
            )
            
            # Удаляем временный файл
            os.unlink(temp_path)
            
            results.append(f"✓ {filename} ({file_size / (1024 * 1024):.1f} MB)")
            
        except Exception as e:
            # Удаляем временный файл в случае ошибки
            if 'temp_path' in locals():
                try:
                    os.unlink(temp_path)
                except:
                    pass
            results.append(f"✗ {filename}: {e}")
    
    return results

# --------- Сравниваем список доступного в MinIO и на сайте на указанный год ---------

def show_current_state(bucket_name="bronze", 
                       prefix="nyc-taxi-data",
                       base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                       filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                       year=2024):
    """Показать текущее состояние файлов"""
    minio_client = Minio(
        "minio:9000",
        access_key="minioadmin", 
        secret_key="minioadmin",
        secure=False
    )
    
    remote_files = get_available_remote_files(base_url, filename_template, year)
    local_files = get_local_minio_files(minio_client, bucket_name, prefix)
    missing_files = list(set(remote_files) - set(local_files))
    
    print("\n📊 ТЕКУЩЕЕ СОСТОЯНИЕ:")
    print(f"• Загружено в MinIO: {len(local_files)} файл(ов)")

    print()
    print(f"• Доступно на сайте: {len(remote_files)} файл(ов)")
    for file in sorted(remote_files):
        print(f"     - {file}")    

    print()
    print(f"• Из них отсутствует в MinIO: {len(missing_files)} файл(ов)")
    if missing_files:
        for file in sorted(missing_files):
            print(f"     - {file}")

In [3]:
# Передаем параметры источника и получателя
results = download_missing_files(
    bucket_name="bronze", 
    prefix="nyc-taxi-data",
    base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
    filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
    year=2025
)

print("Результаты дозагрузки:")
for result in sorted(results):
    print(result)

🔍 Проверка доступных файлов на сайте...


Проверка месяца:  17%|█▋        | 2/12 [00:00<00:00, 11.57it/s]

  ✓ yellow_tripdata_2025-01.parquet - доступен
  ✓ yellow_tripdata_2025-02.parquet - доступен
  ✓ yellow_tripdata_2025-03.parquet - доступен


Проверка месяца:  33%|███▎      | 4/12 [00:00<00:00, 11.35it/s]

  ✓ yellow_tripdata_2025-04.parquet - доступен
  ✓ yellow_tripdata_2025-05.parquet - доступен


...

  ✗ yellow_tripdata_2025-10.parquet - недоступен (код: 403)


Проверка месяца:  92%|█████████▏| 11/12 [00:01<00:00,  4.43it/s]

  ✗ yellow_tripdata_2025-11.parquet - недоступен (код: 403)


Проверка месяца: 100%|██████████| 12/12 [00:02<00:00,  5.32it/s]

  ✗ yellow_tripdata_2025-12.parquet - недоступен (код: 403)

📊 СТАТИСТИКА:
• Загружено в MinIO: 45 файл(ов)

• Доступно на сайте: 9 файл(ов)
     - yellow_tripdata_2025-01.parquet
     - yellow_tripdata_2025-02.parquet
     - yellow_tripdata_2025-03.parquet
     - yellow_tripdata_2025-04.parquet
     - yellow_tripdata_2025-05.parquet
     - yellow_tripdata_2025-06.parquet
     - yellow_tripdata_2025-07.parquet
     - yellow_tripdata_2025-08.parquet
     - yellow_tripdata_2025-09.parquet

Результаты дозагрузки:
✅ Все доступные файлы уже загружены


In [2]:
# Сравниваем список доступного в MinIO и на сайте на указанный год 
show_current_state(bucket_name="bronze", 
                   prefix="nyc-taxi-data",
                   base_url="https://d37ci6vzurychx.cloudfront.net/trip-data",
                   filename_template="yellow_tripdata_{year}-{month:02d}.parquet",
                   year=2025)

🔍 Проверка доступных файлов на сайте...


Проверка месяца:   8%|▊         | 1/12 [00:00<00:01,  5.64it/s]

  ✓ yellow_tripdata_2025-01.parquet - доступен


Проверка месяца:  17%|█▋        | 2/12 [00:00<00:01,  7.26it/s]

  ✓ yellow_tripdata_2025-02.parquet - доступен


...

  ✗ yellow_tripdata_2025-10.parquet - недоступен (код: 403)


Проверка месяца:  92%|█████████▏| 11/12 [00:03<00:00,  2.59it/s]

  ✗ yellow_tripdata_2025-11.parquet - недоступен (код: 403)


Проверка месяца: 100%|██████████| 12/12 [00:03<00:00,  3.30it/s]

  ✗ yellow_tripdata_2025-12.parquet - недоступен (код: 403)

📊 ТЕКУЩЕЕ СОСТОЯНИЕ:
• Загружено в MinIO: 45 файл(ов)

• Доступно на сайте: 9 файл(ов)
     - yellow_tripdata_2025-01.parquet
     - yellow_tripdata_2025-02.parquet
     - yellow_tripdata_2025-03.parquet
     - yellow_tripdata_2025-04.parquet
     - yellow_tripdata_2025-05.parquet
     - yellow_tripdata_2025-06.parquet
     - yellow_tripdata_2025-07.parquet
     - yellow_tripdata_2025-08.parquet
     - yellow_tripdata_2025-09.parquet

• Из них отсутствует в MinIO: 0 файл(ов)


# Обработка данных

Данные за разные года приходят с разной структурой.

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

drivers = [
    "/home/jovyan/work/spark-jars/hadoop-aws-3.3.4.jar",             # S3
    "/home/jovyan/work/spark-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/home/jovyan/work/spark-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/home/jovyan/work/spark-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

25/11/10 19:41:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
df_2022 = spark.read.format("parquet").load("s3a://bronze/nyc-taxi-data/yellow_tripdata_2022-01.parquet")
df_2024 = spark.read.format("parquet").load("s3a://bronze/nyc-taxi-data/yellow_tripdata_2024-01.parquet")

In [ ]:
df_2022.select("VendorID", "passenger_count", "trip_distance", "RatecodeID", "PULocationID").printSchema()
df_2024.select("VendorID", "passenger_count", "trip_distance", "RatecodeID", "PULocationID").printSchema()

# Нормализация типов, загрузка в `silver`

Для того чтобы можно было загружать данные как единый массив напишем функции нормализации.

Функции проверяют уже обработанные срезы, сравнивают с новыми которые появились в Bronze и обрабатывают только новые срехы

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, IntegerType
import re
from minio import Minio
from minio.error import S3Error
import time

# Конфигурация MinIO
MINIO_ENDPOINT = 'minio:9000'  # замените на ваш endpoint
MINIO_ACCESS_KEY = 'minioadmin'
MINIO_SECRET_KEY = 'minioadmin'

def get_minio_client():
    """Создает и возвращает клиент MinIO"""
    return Minio(
        MINIO_ENDPOINT,
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=False  # используйте True для HTTPS
    )

def extract_month_from_filename(file_path):
    """Извлекает месяц из имени файла в формате YYYY-MM"""
    match = re.search(r'(\d{4}-\d{2})', file_path)
    return match.group(1) if match else None

def get_processed_slices(output_bucket, output_prefix):
    """Возвращает список уже обработанных срезов из выходного бакета используя MinIO"""
    try:
        client = get_minio_client()
        processed_slices = set()
        
        # Получаем список объектов в указанном префиксе
        objects = client.list_objects(output_bucket, prefix=output_prefix, recursive=True)
        
        for obj in objects:
            # Извлекаем месяц из пути
            month = extract_month_from_filename(obj.object_name)
            if month:
                processed_slices.add(month)
        
        print(f"📁 Найдено обработанных срезов в {output_bucket}/{output_prefix}: {len(processed_slices)}")
        return processed_slices
        
    except S3Error as e:
        if e.code == 'NoSuchBucket':
            print(f"⚠️ Бакет {output_bucket} не существует или пустой")
        else:
            print(f"⚠️ Ошибка при чтении {output_bucket} бакета: {e}")
        return set()
    except Exception as e:
        print(f"⚠️ Не удалось прочитать {output_bucket} бакет: {e}")
        return set()

def get_input_files_with_months(input_bucket, input_prefix):
    """Возвращает список файлов/папок из входного бакета с извлеченными месяцами используя MinIO"""
    try:
        client = get_minio_client()
        input_files = []
        
        # Получаем список объектов на ПЕРВОМ уровне вложенности (не рекурсивно!)
        objects = client.list_objects(input_bucket, prefix=input_prefix, recursive=False)
        
        for obj in objects:
            object_name = obj.object_name
            
            # Определяем тип объекта: файл .parquet или папка
            is_parquet_file = object_name.endswith('.parquet')
            is_folder = not is_parquet_file and object_name.endswith('/')
            
            if is_parquet_file or is_folder:
                month = extract_month_from_filename(object_name)
                if month:
                    # Формируем S3 путь для Spark
                    s3_path = f"s3a://{input_bucket}/{object_name}"
                    input_files.append({
                        'path': s3_path,
                        'month': month,
                        'file_name': object_name.split('/')[-1] if is_parquet_file else object_name.split('/')[-2] + '/'
                    })
        
        print(f"📁 Найдено объектов в {input_bucket}/{input_prefix}: {len(input_files)}")
        return input_files
        
    except Exception as e:
        print(f"❌ Ошибка при чтении {input_bucket} бакета: {e}")
        return []

def standardize_nyc_taxi_data(input_path, output_path):
    """
    Стандартизирует данные NYC Taxi (ваша существующая функция без изменений)
    """
    output_path = output_path.replace('.parquet', '')
    
    # Читаем исходные данные
    df = spark.read.parquet(input_path)
    
    # 1. Приводим все названия колонок к нижнему регистру
    for col_name in df.columns:
        df = df.withColumnRenamed(col_name, col_name.lower())
    
    # 2. Определяем маппинг типов для унификации
    type_mapping = {
        "vendorid": IntegerType(),
        "pulocationid": IntegerType(), 
        "dolocationid": IntegerType(),
        "payment_type": IntegerType(),
        "ratecodeid": IntegerType(),
        "passenger_count": IntegerType(),
        "fare_amount": DoubleType(),
        "extra": DoubleType(),
        "mta_tax": DoubleType(),
        "tip_amount": DoubleType(),
        "tolls_amount": DoubleType(),
        "improvement_surcharge": DoubleType(),
        "total_amount": DoubleType(),
        "congestion_surcharge": DoubleType(),
        "airport_fee": DoubleType(),
        "cbd_congestion_fee": DoubleType(),
        "trip_distance": DoubleType()
    }
    
    # 3. Применяем преобразование типов с обработкой ошибок
    for col_name, target_type in type_mapping.items():
        if col_name in df.columns:
            df = df.withColumn(
                col_name, 
                F.coalesce(
                    F.col(col_name).cast(target_type), 
                    F.lit(0 if target_type == IntegerType() else 0.0)
                )
            )
    
    # 4. Гарантируем порядок колонок для consistency
    expected_columns = [
        "vendorid", "tpep_pickup_datetime", "tpep_dropoff_datetime",
        "passenger_count", "trip_distance", "ratecodeid", "store_and_fwd_flag",
        "pulocationid", "dolocationid", "payment_type", "fare_amount", "extra",
        "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge",
        "total_amount", "congestion_surcharge", "airport_fee", "cbd_congestion_fee"
    ]
    
    # Выбираем только существующие колонки в правильном порядке
    final_columns = [col for col in expected_columns if col in df.columns]
    df_standardized = df.select(final_columns)
    
    # 5. Сохраняем с оптимальными настройками
    (df_standardized
     .coalesce(1)
     .write
     .mode("overwrite")
     .option("compression", "snappy")
     .parquet(output_path)
    )
    
    print(f"✅ Стандартизировано: {input_path} -> {output_path}")
    return df_standardized

def process_incremental_nyc_taxi_files(input_bucket, input_prefix, output_bucket, output_prefix):
    """Обрабатывает только новые файлы NYC Taxi из входного бакета в выходной"""
    
    # Получаем списки обработанных и доступных файлов через MinIO
    processed_slices = get_processed_slices(output_bucket, output_prefix)
    input_files = get_input_files_with_months(input_bucket, input_prefix)
    
    # Фильтруем только новые файлы
    new_files = [f for f in input_files if f['month'] not in processed_slices]
    
    print(f"📊 Статистика:")
    print(f"   - Всего во входном бакете: {len(input_files)}")
    print(f"   - Уже в выходном бакете: {len(processed_slices)}") 
    print(f"   - Новых для обработки: {len(new_files)}")
    
    if not new_files:
        print("🎉 Все срезы уже обработаны! Ничего делать не нужно.")
        return
    
    # Обрабатываем только новые файлы
    for i, file_info in enumerate(new_files, 1):
        input_path = file_info['path']
        file_name = file_info['file_name']
        
        # Формируем выходной путь, сохраняя структуру после префикса
        # Пример: входной путь s3a://bronze/nyc-taxi-data/yellow_tripdata_2022-01
        # Выходной путь: s3a://silver/nyc-taxi-data/yellow_tripdata_2022-01
        output_path = f"s3a://{output_bucket}/{output_prefix}{file_name}".replace('.parquet', '')
        
        print(f"🔄 Обрабатываю новый срез ({i}/{len(new_files)}): {file_info['month']}")
        
        try:
            standardize_nyc_taxi_data(input_path, output_path)
            print(f"✅ Успешно обработан: {file_info['month']}")
        except Exception as e:
            print(f"❌ Ошибка при обработке {file_info['month']}: {e}")
    
    print(f"🎉 Обработка завершена! Обработано {len(new_files)} новых срезов.")

# Дополнительная функция для отладки - просмотр структуры бакетов
def inspect_buckets(bucket_name, prefix):
    """Функция для отладки - показывает структуру бакетов"""
    try:
        client = get_minio_client()
        
        print(f"🔍 Инспекция бакета {bucket_name}/{prefix}:")
        objects = client.list_objects(bucket_name, prefix=prefix, recursive=True)
        for obj in objects:
            print(f"   {bucket_name}: {obj.object_name}")
            
    except Exception as e:
        print(f"❌ Ошибка при инспекции бакета {bucket_name}: {e}")

# Пример использования:
# process_incremental_nyc_taxi_files('bronze', 'nyc-taxi-data/', 'silver', 'nyc-taxi-data/')

In [6]:
# Обработка данных из bronze в silver
process_incremental_nyc_taxi_files(
    input_bucket='bronze', 
    input_prefix='nyc-taxi-data/',
    output_bucket='silver', 
    output_prefix='nyc-taxi-data-norm/'
)

📁 Найдено обработанных срезов в silver/nyc-taxi-data-norm/: 45
📁 Найдено объектов в bronze/nyc-taxi-data/: 45
📊 Статистика:
   - Всего во входном бакете: 45
   - Уже в выходном бакете: 45
   - Новых для обработки: 0
🎉 Все срезы уже обработаны! Ничего делать не нужно.


In [ ]:
# Для отладки можно посмотреть структуру бакетов
inspect_buckets('bronze', 'nyc-taxi-data/')
inspect_buckets('silver', 'nyc-taxi-data-norm/')

# Работа со слоем Silver

# Предобработка

In [ ]:
df = spark.read.format("parquet").load("s3a://silver/nyc-taxi-data-norm/yellow_tripdata_2025-01")

# Кэшируем, т.к. будем делать несколько действий
# df.cache()

# Базовая информация
print("Количество строк и столбцов:", (df.count(), len(df.columns)))
df.printSchema()

### Шаг 1: Анализ Пропущенных Значений

In [ ]:
# from pyspark.sql.functions import col, sum, count

# Считаем пропуски для каждой колонки
total_rows = df.count()
null_counts = []

for col_name in df.columns:
    null_count = df.where(F.col(col_name).isNull()).count()
    null_percentage = (null_count / total_rows) * 100
    null_counts.append((col_name, null_count, null_percentage))

# Создаем датафрейм для удобного просмотра
nulls_df = spark.createDataFrame(
    null_counts,
    ["column_name", "null_count", "null_percentage"]
).orderBy(F.desc("null_count"))

nulls_df.show()

Решение:
- Удаляем поле `store_and_fwd_flag`

### Шаг 2: Анализ Аномальных и Некорректных Значений

In [ ]:
df.groupBy("passenger_count").count().orderBy("passenger_count").show()

Решение:
- Удаляем значения пассажиров `> 6`

In [ ]:
# Быстрая статистика
df.select(
    F.mean("trip_distance").alias("mean"),
    F.stddev("trip_distance").alias("stddev"),
    F.min("trip_distance").alias("min"),
    F.max("trip_distance").alias("max")
).show()

# Считаем приблизительные квантили для определения выбросов
quantiles = df.approxQuantile("trip_distance", [0.01, 0.25, 0.5, 0.75, 0.99], 0.05)
print(f"Квантили расстояния: 1%: {quantiles[0]}, 25%: {quantiles[1]}, 50%: {quantiles[2]}, 75%: {quantiles[3]}, 99%: {quantiles[4]}")

In [ ]:
df.select("trip_distance").filter(F.col("trip_distance") > 100).count()

Решение: 
- Удаляем нулевые поездки
- Поездки более 100 км.

In [ ]:
# Правильное создание df_with_duration
df_with_duration = df.withColumn(
    "trip_duration_minutes",
    (F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")) / 60
)

# Проверяем, что получилось
df_with_duration.select("tpep_pickup_datetime", "tpep_dropoff_datetime", "trip_duration_minutes").show(5)

# Теперь статистики должны работать
df_with_duration.select(
    F.mean("trip_duration_minutes").alias("mean_duration"),
    F.stddev("trip_duration_minutes").alias("stddev_duration"),
    F.min("trip_duration_minutes").alias("min_duration"),
    F.max("trip_duration_minutes").alias("max_duration")
).show()

In [ ]:
df_with_duration.filter(F.col("trip_duration_minutes") > 90).count()

Решение:
- Удаляем значения `<= 0`
- Удаляем значения `> 90`

In [ ]:
# Пример для total_amount
payment_cols = ["fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount", "improvement_surcharge", "total_amount"]
for col_name in payment_cols:
    stats = df.select(
        F.mean(col_name).alias("mean"),
        F.min(col_name).alias("min"),
        F.max(col_name).alias("max")
    ).collect()[0]
    print(f"{col_name}: min={stats['min']}, max={stats['max']}, mean={stats['mean']:.2f}")

    # Квантили для ключевых колонок
    if col_name in ["fare_amount", "total_amount"]:
        quantiles = df.approxQuantile(col_name, [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99], 0.05)
        print(f"  Квантили: {quantiles}")

In [ ]:
df.select("trip_distance", "fare_amount", "total_amount").filter(F.col("tolls_amount") > 30).count()

Решение:
- Удаляем `total_amount > 110`

In [ ]:
# Рассчитываем IQR для trip_duration_minutes
q1, q3 = df_with_duration.approxQuantile("trip_duration_minutes", [0.25, 0.75], 0.05)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

print(f"Границы для длительности поездки: Lower={lower_bound:.2f}, Upper={upper_bound:.2f}")

# Аналогично для total_amount
q1_t, q3_t = df.approxQuantile("total_amount", [0.25, 0.75], 0.05)
iqr_t = q3_t - q1_t
calculated_lower = q1_t - 1.5 * iqr_t
lower_bound_t = 0 if calculated_lower < 0 else calculated_lower  # Используем тернарный оператор
upper_bound_t = q3_t + 1.5 * iqr_t

print(f"Границы для общей суммы: Lower={lower_bound_t:.2f}, Upper={upper_bound_t:.2f}")

### Собираем итоговый датафрейм

In [ ]:
# Создаем отфильтрованный датафрейм
df_clean = df_with_duration.filter(
    # Временные аномалии: длительность больше 1 минуты и меньше верхней границы
    (F.col("passenger_count") >= 0) &
    (F.col("passenger_count") <= 6) &

    (F.col("trip_duration_minutes") > 1) &
    (F.col("trip_duration_minutes") < 90) &

    # Платежи
    (F.col("fare_amount") > 0) &
    (F.col("fare_amount") < 110) &
    (F.col("extra") >= 0) &
    (F.col("mta_tax") >= 0) &
    (F.col("improvement_surcharge") >= 0) &
    (F.col("tip_amount") >= 0) &
    (F.col("tip_amount") < 30) &
    (F.col("tolls_amount") >= 0) &
    (F.col("tolls_amount") < 30) &    
    (F.col("total_amount") > 0) &
    (F.col("total_amount") < 110) &

    (F.col("trip_distance") > 0) &
    (F.col("trip_distance") < 100) & 

    (F.year("tpep_pickup_datetime") >= 2022) &
    (F.year("tpep_pickup_datetime") < 2026)
    # Добавьте другие условия по необходимости
)



# Обогащаем

df_clean = (df_clean
            .withColumn("year",        F.year("tpep_pickup_datetime")) # Год
            .withColumn("month",       F.month("tpep_pickup_datetime")) # Месяц
            .withColumn("day",         F.dayofmonth("tpep_pickup_datetime"))
            .withColumn("day_of_week", F.dayofweek("tpep_pickup_datetime"))
            .withColumn("hour",        F.hour("tpep_pickup_datetime"))
            .withColumn("is_weekend",  F.when(F.dayofweek("tpep_pickup_datetime").isin(1, 7), 1).otherwise(0))
            .withColumn("time_of_day",
                        F.when((F.col("hour") >= 5) &  (F.col("hour") < 12), "Утро")
                         .when((F.col("hour") >= 12) & (F.col("hour") < 17), "День")
                         .when((F.col("hour") >= 17) & (F.col("hour") < 21), "Вечер")
                         .otherwise("Ночь"))
            .withColumn("is_rush_hour",
                        F.when(
                            ((F.col("hour") >= 7)  & (F.col("hour") <= 10)) | # Утренний пик
                            ((F.col("hour") >= 16) & (F.col("hour") <= 19)),  # Вечерний пик
                            1
                        ).otherwise(0))
            .withColumn("avg_speed_kmh",
                        F.when(F.col("trip_duration_minutes") > 0,
                               (F.col("trip_distance") * 1.60934) / (F.col("trip_duration_minutes") / 60.0)
                              ).otherwise(None))
            .withColumn("tip_ratio", F.col("tip_amount") / F.col("fare_amount"))
            .withColumn("has_tip",   F.when(F.col("tip_amount") > 0, 1).otherwise(0)) # Бинарный целевой признак
            .withColumn("revenue_per_minute",
                        F.when(F.col("trip_duration_minutes") > 0,
                               F.col("total_amount") / F.col("trip_duration_minutes")
                              ).otherwise(None))
           )
# Удаляем поля с большим количеством пропусков
df_clean = df_clean.drop("store_and_fwd_flag")



print(f"Исходный размер: {df.count()}")
print(f"Размер после очистки: {df_clean.count()}")
print(f"Удалено {df.count() - df_clean.count()} строк ({(1 - df_clean.count()/df.count())*100:.2f}%)")

In [ ]:
df_clean.printSchema()

In [ ]:
df_clean.show(1)

# Функция очистки

In [ ]:
df = spark.read.format("parquet").load("s3a://silver/nyc-taxi-data/yellow_tripdata_2025-01")

df_with_duration = df.withColumn(
    "trip_duration_minutes",
    (F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")) / 60
)

# Создаем отфильтрованный датафрейм
df_clean = df_with_duration.filter(
    # Временные аномалии: длительность больше 1 минуты и меньше верхней границы
    (F.col("passenger_count") >= 0) &
    (F.col("passenger_count") <= 6) &

    (F.col("trip_duration_minutes") > 1) &
    (F.col("trip_duration_minutes") < 90) &

    # Платежи
    (F.col("fare_amount") > 0) &
    (F.col("fare_amount") < 110) &
    (F.col("extra") >= 0) &
    (F.col("mta_tax") >= 0) &
    (F.col("improvement_surcharge") >= 0) &
    (F.col("tip_amount") >= 0) &
    (F.col("tip_amount") < 30) &
    (F.col("tolls_amount") >= 0) &
    (F.col("tolls_amount") < 30) &    
    (F.col("total_amount") > 0) &
    (F.col("total_amount") < 110) &

    (F.col("trip_distance") > 0) &
    (F.col("trip_distance") < 100) & 

    (F.year("tpep_pickup_datetime") >= 2022) &
    (F.year("tpep_pickup_datetime") < 2026)
    # Добавьте другие условия по необходимости
)



# Обогащаем

df_clean = (df_clean
            .withColumn("year",        F.year("tpep_pickup_datetime")) # Год
            .withColumn("month",       F.month("tpep_pickup_datetime")) # Месяц
            .withColumn("day",         F.dayofmonth("tpep_pickup_datetime"))
            .withColumn("day_of_week", F.dayofweek("tpep_pickup_datetime"))
            .withColumn("hour",        F.hour("tpep_pickup_datetime"))
            .withColumn("is_weekend",  F.when(F.dayofweek("tpep_pickup_datetime").isin(1, 7), 1).otherwise(0))
            .withColumn("time_of_day",
                        F.when((F.col("hour") >= 5) &  (F.col("hour") < 12), "Утро")
                         .when((F.col("hour") >= 12) & (F.col("hour") < 17), "День")
                         .when((F.col("hour") >= 17) & (F.col("hour") < 21), "Вечер")
                         .otherwise("Ночь"))
            .withColumn("is_rush_hour",
                        F.when(
                            ((F.col("hour") >= 7)  & (F.col("hour") <= 10)) | # Утренний пик
                            ((F.col("hour") >= 16) & (F.col("hour") <= 19)),  # Вечерний пик
                            1
                        ).otherwise(0))
            .withColumn("avg_speed_kmh",
                        F.when(F.col("trip_duration_minutes") > 0,
                               (F.col("trip_distance") * 1.60934) / (F.col("trip_duration_minutes") / 60.0)
                              ).otherwise(None))
            .withColumn("tip_ratio", F.col("tip_amount") / F.col("fare_amount"))
            .withColumn("has_tip",   F.when(F.col("tip_amount") > 0, 1).otherwise(0)) # Бинарный целевой признак
            .withColumn("revenue_per_minute",
                        F.when(F.col("trip_duration_minutes") > 0,
                               F.col("total_amount") / F.col("trip_duration_minutes")
                              ).otherwise(None))
           )
# Удаляем поля с большим количеством пропусков
df_clean = df_clean.drop("store_and_fwd_flag")



print(f"Исходный размер: {df.count()}")
print(f"Размер после очистки: {df_clean.count()}")
print(f"Удалено {df.count() - df_clean.count()} строк ({(1 - df_clean.count()/df.count())*100:.2f}%)")

In [ ]:
spark.stop()

In [7]:

def eda_nyc_taxi_data(input_path, output_path):
    """
    Очищает данные NYC Taxi 
    """
    df = spark.read.format("parquet").load(input_path)
    
    df_with_duration = df.withColumn(
        "trip_duration_minutes",
        (F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")) / 60
    )
    
    # Создаем отфильтрованный датафрейм
    df_clean = df_with_duration.filter(
        # Временные аномалии: длительность больше 1 минуты и меньше верхней границы
        (F.col("passenger_count") >= 0) &
        (F.col("passenger_count") <= 6) &
    
        (F.col("trip_duration_minutes") > 1) &
        (F.col("trip_duration_minutes") < 90) &
    
        # Платежи
        (F.col("fare_amount") > 0) &
        (F.col("fare_amount") < 110) &
        (F.col("extra") >= 0) &
        (F.col("mta_tax") >= 0) &
        (F.col("improvement_surcharge") >= 0) &
        (F.col("tip_amount") >= 0) &
        (F.col("tip_amount") < 30) &
        (F.col("tolls_amount") >= 0) &
        (F.col("tolls_amount") < 30) &    
        (F.col("total_amount") > 0) &
        (F.col("total_amount") < 110) &
    
        (F.col("trip_distance") > 0) &
        (F.col("trip_distance") < 100) & 
    
        (F.year("tpep_pickup_datetime") >= 2022) &
        (F.year("tpep_pickup_datetime") < 2026)
        # Добавьте другие условия по необходимости
    )
    
    
    
    # Обогащаем
    
    df_clean = (df_clean
                .withColumn("date_month",  F.date_trunc("month", "tpep_pickup_datetime"))
                .withColumn("year",        F.year("tpep_pickup_datetime")) # Год
                .withColumn("month",       F.month("tpep_pickup_datetime")) # Месяц
                .withColumn("day",         F.dayofmonth("tpep_pickup_datetime"))
                .withColumn("day_of_week", F.dayofweek("tpep_pickup_datetime"))
                .withColumn("hour",        F.hour("tpep_pickup_datetime"))
                .withColumn("is_weekend",  F.when(F.dayofweek("tpep_pickup_datetime").isin(1, 7), 1).otherwise(0))
                .withColumn("time_of_day",
                            F.when((F.col("hour") >= 5) &  (F.col("hour") < 12), "Утро")
                             .when((F.col("hour") >= 12) & (F.col("hour") < 17), "День")
                             .when((F.col("hour") >= 17) & (F.col("hour") < 21), "Вечер")
                             .otherwise("Ночь"))
                .withColumn("is_rush_hour",
                            F.when(
                                ((F.col("hour") >= 7)  & (F.col("hour") <= 10)) | # Утренний пик
                                ((F.col("hour") >= 16) & (F.col("hour") <= 19)),  # Вечерний пик
                                1
                            ).otherwise(0))
                .withColumn("avg_speed_kmh",
                            F.when(F.col("trip_duration_minutes") > 0,
                                   (F.col("trip_distance") * 1.60934) / (F.col("trip_duration_minutes") / 60.0)
                                  ).otherwise(None))
                .withColumn("tip_ratio", F.col("tip_amount") / F.col("fare_amount"))
                .withColumn("has_tip",   F.when(F.col("tip_amount") > 0, 1).otherwise(0)) # Бинарный целевой признак
                .withColumn("revenue_per_minute",
                            F.when(F.col("trip_duration_minutes") > 0,
                                   F.col("total_amount") / F.col("trip_duration_minutes")
                                  ).otherwise(None))
               )
    # Удаляем поля с большим количеством пропусков
    df_clean = df_clean.drop("store_and_fwd_flag")
    
    
    
    print(f"Исходный размер: {df.count()}")
    print(f"Размер после очистки: {df_clean.count()}")
    print(f"Удалено {df.count() - df_clean.count()} строк ({(1 - df_clean.count()/df.count())*100:.2f}%)")
      
   
    # 5. Сохраняем с оптимальными настройками
    (df_clean
     .coalesce(1)
     .write
     .mode("overwrite")
     .option("compression", "snappy")
     .parquet(output_path)
    )
    
    print(f"✅ Стандартизировано: {input_path} -> {output_path}")
    return df_clean

def eda_incremental_nyc_taxi_files(input_bucket, input_prefix, output_bucket, output_prefix):
    """Обрабатывает только новые файлы NYC Taxi из входного бакета в выходной"""
    
    # Получаем списки обработанных и доступных файлов через MinIO
    processed_slices = get_processed_slices(output_bucket, output_prefix)
    input_files = get_input_files_with_months(input_bucket, input_prefix)
    
    # Фильтруем только новые файлы
    new_files = [f for f in input_files if f['month'] not in processed_slices]
    
    print(f"📊 Статистика:")
    print(f"   - Всего во входном бакете: {len(input_files)}")
    print(f"   - Уже в выходном бакете: {len(processed_slices)}") 
    print(f"   - Новых для обработки: {len(new_files)}")
    
    if not new_files:
        print("🎉 Все срезы уже обработаны! Ничего делать не нужно.")
        return
    
    start_time = time.time()
    
    # Обрабатываем только новые файлы
    for i, file_info in enumerate(new_files, 1):
        input_path = file_info['path']
        file_name = file_info['file_name']
        
        # Формируем выходной путь, сохраняя структуру после префикса
        # Пример: входной путь s3a://bronze/nyc-taxi-data/yellow_tripdata_2022-01
        # Выходной путь: s3a://silver/nyc-taxi-data/yellow_tripdata_2022-01
        output_path = f"s3a://{output_bucket}/{output_prefix}{file_name}".replace('.parquet', '')
        
        print(f"🔄 Обрабатываю новый срез ({i}/{len(new_files)}): {file_info['month']}")
        
        try:
            eda_nyc_taxi_data(input_path, output_path)
            print(f"✅ Успешно обработан: {file_info['month']}")
        except Exception as e:
            print(f"❌ Ошибка при обработке {file_info['month']}: {e}")
    
    end_time = time.time()
    elapsed = end_time - start_time
    print(f"🎉 Обработка завершена! Обработано {len(new_files)} новых срезов.")
    print(f"Код выполнялся: {elapsed:.3f} сек.")

In [8]:
# Обработка данных из norm в eda
eda_incremental_nyc_taxi_files(
    input_bucket='silver', 
    input_prefix='nyc-taxi-data-norm/',
    output_bucket='silver', 
    output_prefix='nyc-taxi-data-eda/'
)

📁 Найдено обработанных срезов в silver/nyc-taxi-data-eda/: 45
📁 Найдено объектов в silver/nyc-taxi-data-norm/: 45
📊 Статистика:
   - Всего во входном бакете: 45
   - Уже в выходном бакете: 45
   - Новых для обработки: 0
🎉 Все срезы уже обработаны! Ничего делать не нужно.


In [9]:
df = spark.read.parquet("s3a://silver/nyc-taxi-data-eda/*") #yellow_tripdata_2025-09/")

25/11/10 19:47:28 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
                                                                                

In [10]:
df.groupBy("date_month", "year", "month").count().orderBy("year", "month").show(50)

+-------------------+----+-----+-------+
|         date_month|year|month|  count|
+-------------------+----+-----+-------+
|2022-01-01 00:00:00|2022|    1|2405105|
|2022-02-01 00:00:00|2022|    2|2911596|
|2022-03-01 00:00:00|2022|    3|3540060|
|2022-04-01 00:00:00|2022|    4|3509368|
|2022-05-01 00:00:00|2022|    5|3487114|
|2022-06-01 00:00:00|2022|    6|3447089|
|2022-07-01 00:00:00|2022|    7|3081872|
|2022-08-01 00:00:00|2022|    8|3056951|
|2022-09-01 00:00:00|2022|    9|3080827|
|2022-10-01 00:00:00|2022|   10|3556349|
|2022-11-01 00:00:00|2022|   11|3140479|
|2022-12-01 00:00:00|2022|   12|3277500|
|2023-01-01 00:00:00|2023|    1|2971693|
|2023-02-01 00:00:00|2023|    2|2823988|
|2023-03-01 00:00:00|2023|    3|3293502|
|2023-04-01 00:00:00|2023|    4|3184656|
|2023-05-01 00:00:00|2023|    5|3396540|
|2023-06-01 00:00:00|2023|    6|3192332|
|2023-07-01 00:00:00|2023|    7|2795870|
|2023-08-01 00:00:00|2023|    8|2706311|
|2023-09-01 00:00:00|2023|    9|2681346|
|2023-10-01 00:0

In [11]:
df.printSchema()

root
 |-- vendorid: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- ratecodeid: integer (nullable = true)
 |-- pulocationid: integer (nullable = true)
 |-- dolocationid: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- trip_duration_minutes: double (nullable = true)
 |-- date_month: timestamp (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = 

# Агрегаты

In [ ]:
df.groupBy("hour").agg(
    F.count("*").alias("total_trips"),
    F.avg("trip_distance").alias("avg_distance"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("avg_speed_kmh").alias("avg_speed"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.avg(F.col("has_tip").cast("double")).alias("tip_probability"),
    F.sum("total_amount").alias("total_revenue")
).orderBy("hour").show(30)

In [ ]:
df.groupBy("day_of_week", "is_weekend").agg(
    F.count("*").alias("total_trips"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("passenger_count").alias("avg_passengers")
).orderBy("day_of_week").show()

In [ ]:
df.groupBy("time_of_day").agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("avg_speed_kmh").alias("avg_speed"),
    F.avg("tip_ratio").alias("avg_tip_ratio")
).show()

In [ ]:
df.filter(F.col("avg_speed_kmh") > 0).groupBy(
    F.when(F.col("avg_speed_kmh") < 10, "0-10 kmh")
     .when(F.col("avg_speed_kmh") < 20, "10-20 kmh") 
     .when(F.col("avg_speed_kmh") < 30, "20-30 kmh")
     .otherwise("30+ kmh").alias("speed_bucket")
).agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("revenue_per_minute").alias("avg_revenue_per_min")
).show()

In [ ]:
df.groupBy(
    F.when(F.col("trip_distance") < 1, "0-1 mi")
     .when(F.col("trip_distance") < 3, "1-3 mi")
     .when(F.col("trip_distance") < 5, "3-5 mi")
     .when(F.col("trip_distance") < 10, "5-10 mi")
     .otherwise("10+ mi").alias("distance_bucket")
).agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("tip_ratio").alias("avg_tip_ratio")
).show()

In [ ]:
df.groupBy("payment_type").agg(
    F.count("*").alias("transaction_count"),
    F.avg("total_amount").alias("avg_amount"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.avg(F.col("has_tip").cast("double")).alias("tip_probability")
).show()

Отличная структура! Вот комплексные агрегаты для анализа и дашбордов, сгруппированные по ключевым темам.

### 1. Временные агрегаты (по часам, дням недели, weekend/weekday)

```python
# Агрегаты по часам
time_hourly_agg = df.groupBy("hour").agg(
    F.count("*").alias("total_trips"),
    F.avg("trip_distance").alias("avg_distance"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("avg_speed_kmh").alias("avg_speed"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.avg(F.col("has_tip").cast("double")).alias("tip_probability"),
    F.sum("total_amount").alias("total_revenue")
).orderBy("hour")

# Агрегаты по дням недели
daily_agg = df.groupBy("day_of_week", "is_weekend").agg(
    F.count("*").alias("total_trips"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("passenger_count").alias("avg_passengers")
).orderBy("day_of_week")
```

### 2. Географические агрегаты (по зонам посадки/высадки)

```python
# Топ зон посадки
top_pickup_locations = df.groupBy("pulocationid").agg(
    F.count("*").alias("pickup_count"),
    F.avg("total_amount").alias("avg_fare_from_zone"),
    F.avg("trip_duration_minutes").alias("avg_duration_from_zone"),
    F.avg("tip_ratio").alias("avg_tip_ratio_from_zone")
).orderBy(F.desc("pickup_count"))

# Топ маршрутов (путь из A в B)
top_routes = df.groupBy("pulocationid", "dolocationid").agg(
    F.count("*").alias("route_popularity"),
    F.avg("trip_distance").alias("avg_route_distance"),
    F.avg("total_amount").alias("avg_route_revenue"),
    F.avg("trip_duration_minutes").alias("avg_route_duration")
).orderBy(F.desc("route_popularity"))
```

### 3. Агрегаты по типам поездок и тарифам

```python
# По ratecodeid (тип поездки)
ratecode_agg = df.groupBy("ratecodeid").agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_amount"),
    F.avg("trip_distance").alias("avg_distance"),
    F.avg("trip_duration_minutes").alias("avg_duration")
)

# Анализ по времени суток
time_of_day_agg = df.groupBy("time_of_day").agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("avg_speed_kmh").alias("avg_speed"),
    F.avg("tip_ratio").alias("avg_tip_ratio")
)
```

### 4. Анализ эффективности и доходности

```python
# Агрегаты по скорости и дистанции
speed_analysis = df.filter(F.col("avg_speed_kmh") > 0).groupBy(
    F.when(F.col("avg_speed_kmh") < 10, "0-10 kmh")
     .when(F.col("avg_speed_kmh") < 20, "10-20 kmh") 
     .when(F.col("avg_speed_kmh") < 30, "20-30 kmh")
     .otherwise("30+ kmh").alias("speed_bucket")
).agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("revenue_per_minute").alias("avg_revenue_per_min")
)

# Анализ по расстоянию
distance_analysis = df.groupBy(
    F.when(F.col("trip_distance") < 1, "0-1 mi")
     .when(F.col("trip_distance") < 3, "1-3 mi")
     .when(F.col("trip_distance") < 5, "3-5 mi")
     .when(F.col("trip_distance") < 10, "5-10 mi")
     .otherwise("10+ mi").alias("distance_bucket")
).agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("tip_ratio").alias("avg_tip_ratio")
)
```

### 5. Анализ чаевых и платежей

```python
# Анализ чаевых по времени и типу оплаты
tip_analysis = df.groupBy("payment_type", "time_of_day").agg(
    F.count("*").alias("total_transactions"),
    F.sum("has_tip").alias("tipped_transactions"),
    F.avg("tip_amount").alias("avg_tip_amount"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    (F.sum("has_tip") / F.count("*")).alias("tip_frequency")
)

# Распределение по типам платежей
payment_analysis = df.groupBy("payment_type").agg(
    F.count("*").alias("transaction_count"),
    F.avg("total_amount").alias("avg_amount"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.avg(F.col("has_tip").cast("double")).alias("tip_probability")
)
```

### 6. Агрегаты по водителям/провайдерам

```python
# Сравнение вендоров
vendor_analysis = df.groupBy("vendorid").agg(
    F.count("*").alias("total_trips"),
    F.avg("total_amount").alias("avg_revenue_per_trip"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("passenger_count").alias("avg_passengers"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.sum("total_amount").alias("total_revenue")
)
```

### 7. Комплексные кросс-агрегаты

```python
# Время суток + день недели
cross_time_analysis = df.groupBy("time_of_day", "day_of_week").agg(
    F.count("*").alias("trip_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("revenue_per_minute").alias("efficiency")
)

# География + время
location_time_analysis = df.groupBy("pulocationid", "time_of_day").agg(
    F.count("*").alias("pickup_count"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration")
)
```

### Рекомендации для дашборда:

**Главные метрики KPI:**
- Общее количество поездок
- Общая выручка
- Средний чек
- Процент поездок с чаевыми
- Средняя скорость

**Основные графики:**
1. **Тепловая карта:** Часы дня × Дни недели (интенсивность поездок)
2. **График линии:** Динамика поездок по часам
3. **Bar chart:** Топ-10 зон посадки
4. **Scatter plot:** Связь расстояния и стоимости
5. **Pie chart:** Распределение по времени суток
6. **Histogram:** Распределение чаевых

Все эти агрегаты можно вычислять инкрементально и обновлять в дашборде при добавлении новых данных.

In [ ]:
df_agg = df.groupBy(
    "date_month", "year", "month", "day_of_week", "time_of_day", 
    "pulocationid"
).agg(
    F.count("*").alias("trip_count"),
    F.sum("total_amount").alias("total_revenue"),
    F.avg("total_amount").alias("avg_revenue"),
    F.avg("trip_duration_minutes").alias("avg_duration"),
    F.avg("trip_distance").alias("avg_distance"),
    F.avg("avg_speed_kmh").alias("avg_speed"),
    F.avg("tip_ratio").alias("avg_tip_ratio"),
    F.avg(F.col("has_tip").cast("double")).alias("tip_probability"),
    F.avg("passenger_count").alias("avg_passengers"),
    F.avg("revenue_per_minute").alias("avg_efficiency")
)

In [ ]:
(df_agg.write.format("jdbc")
         .option("url", "jdbc:postgresql://postgres-db:5432/learn_base")
         .option("driver", "org.postgresql.Driver")
         .option("user", "airflow")
         .option("password", "airflow")
         .option("dbtable", "nyc_taxi.nyc_taxi_agg_table")
         .option("batchsize", 10000)
         .mode("overwrite")
         .save())


## **Варианты графиков для дашборда:**

### **1. Временные тренды**
- **Линейный график:** Динамика поездок/выручки по месяцам (фильтр: конкретная зона или "все")
- **Heatmap:** Дни недели × Время суток (интенсивность поездок)

### **2. Географический анализ**  
- **Карта:** Тепловая карта по зонам (количество поездок/средний чек)
- **Bar chart:** Топ-10 зон по выручке/поездкам

### **3. Сравнительный анализ**
- **Scatter plot:** Эффективность (доход/минуту) vs Чаевые по зонам
- **Grouped bar:** Сравнение метрик по времени суток (утро/день/вечер/ночь)

### **4. Детализация по фильтрам**
- **Все графики** должны обновляться при выборе:
  - Год/Месяц
  - День недели 
  - Время суток
  - Зона посадки

**Преимущества подхода:**
- ✅ Одна таблица на все случаи
- ✅ Быстрые отклики на любые фильтры
- ✅ Возможность drill-down от общего к частному
- ✅ Укладываемся в 400k строк

Что скажете? Такой подход подходит? Какие метрики или разрезы хотите добавить/убрать?

In [ ]:
spark.stop()